In [10]:
import pandas as pd
import csv
from geopy.distance import great_circle

In [11]:
with open('nyc_taxi_data.csv', 'rt') as f:
    reader = csv.reader(f)
    data1 = list(reader)

In [12]:
data = pd.read_csv('nyc_taxi_data.csv')
data = data[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','trip_distance']]

In [13]:
df1 = data[['pickup_latitude', 'pickup_longitude']]
df1 = df1.rename({'pickup_latitude': 'lat', 'pickup_longitude': 'long'}, axis =1)
df2 = data[['dropoff_latitude', 'dropoff_longitude']]
df2 = df2.rename({'dropoff_latitude': 'lat', 'dropoff_longitude': 'long'}, axis = 1)

In [14]:
df3 = pd.concat([df1,df2], ignore_index = True)
#df4 = pd.concat([df1,df2], ignore_index = True)
df3 = df3.drop_duplicates(subset = ['lat','long'])

In [15]:
df3.reset_index(inplace = True)

In [16]:
df3 = df3.rename(columns = {'index':'nodeid'})
df3.to_csv('nodes.csv', index = False)

In [17]:
with open('nodes.csv', 'rt') as f:
    reader = csv.reader(f)
    data3 = list(reader)

In [18]:
#new

taxi_data = pd.read_csv("nyc_taxi_data.csv")            # create taxi_data dataframe
node_data = pd.read_csv("nodes.csv")                    # create node_data dataframe

taxi_data["start_node"] = [-1] * len(taxi_data)         # create a new column start_node and initialize columns with -1
taxi_data["end_node"] = [-1] * len(taxi_data)           # create a new column end_node  and initialize columns with -1

for node in node_data.values:                           # read the nodes in node.csv row-wise

    start_node = node[0]                                # node_id
    pickup_lat = node[1]                                # lat
    pickup_long = node[2]                               # long

    if not taxi_data.loc[ (taxi_data["pickup_latitude"] == pickup_lat ) & (taxi_data["pickup_longitude"] == pickup_long) ].empty:                     # if the node exists in the taxi_data
        index_taxi = taxi_data.loc[ (taxi_data['pickup_latitude'] == pickup_lat) & (taxi_data['pickup_longitude'] == pickup_long) ].index.values[0]   # get the node index
        taxi_data.at[index_taxi,"start_node"] = start_node                                                                  # store the node_id of the starting node in taxi_data
        end_node_lat, end_node_long = taxi_data.loc[index_taxi]["dropoff_latitude"], taxi_data.loc[index_taxi]["dropoff_longitude"]   # get the dropoff latitude and longitude from taxi_data

    index_node = node_data.loc[ (node_data["lat"]==end_node_lat) & (node_data["long"]==end_node_long) ].index.values[0]     # search for the dropoff node in node_data
    taxi_data.at[index_taxi, "end_node"] = node_data.at[index_node, "nodeid"]                                              # store the node_id of the end node in taxi_data

edges = []                                          # list of edges
for trip in taxi_data.values:                       # loop over taxi_data
    edge = [int(trip[5]), int(trip[6])]             # get the start node and end node
    edges.append(edge)                              # append edge to edges list
    edge = [int(trip[6]), int(trip[5])]             # reverse the start node and end node
    edges.append(edge)                              # append to edges list

edges_df = pd.DataFrame(edges, columns= ["node_id1", "node_id2"])   # convert edgest into a dataframe

edges_df.to_csv("edges.csv", index=False)                           # print edges_df dataframe as csv file